# Preliminary analysis of extracted convolutional layers

### includes auto MP fits and mp soft ranks



In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn

import numpy as np
import powerlaw

import sklearn
from sklearn.decomposition import TruncatedSVD

from tqdm import tqdm_notebook as tqdm

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [ ]:
import import_ipynb
import RMT_Util

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [5,5]

### All Linear Models

In [ ]:
def list_layers(model):
    pretrained_model = model(pretrained=True)
    
    model_name = model.__name__
    model_name = model_name.replace("_",' ')
    alphas = []
    soft_ranks = []
    
    for im, m in enumerate(pretrained_model.modules()):
        #print(im,m)
        if m isinstance(m, nn.Linear):     
            W = np.array(m.weight.data.clone().cpu())
            if W is not None:
                print(im,m, W.shape)
        elif m isinstance(m, nn.Conv2d):
            #print("conv2d")
            Wtensor = np.array(m.weight.data.clone().cpu())
            Wmats = get_conv2D_Wmats(Wtensor)
            print(im,m)
            for W in Wmats:
                print(W.shape)
        
        

In [ ]:
def get_conv2D_Wmats(Wtensor):
    """Extract W slices from a 4 index conv2D tensor of shape: (N,M,i,j) or (M,N,i,j).  Return ij (N x M) matrices"""
    Wmats = []
    s = Wtensor.shape
    N, M, imax, jmax = s[0],s[1],s[2],s[3]
    print("tensor shape", N,M,imax,jmax)
    for i in range(imax):
        for j in range(jmax):
            W = Wtensor[:,:,i,j]
            if N < M:
                W = W.T
            Wmats.append(W)
    return Wmats

In [ ]:
def analyze_model(model, linear=False, conv2d=True, plot=True):
    pretrained_model = model(pretrained=True)
    
    model_name = model.__name__
    model_name = model_name.replace("_",' ')
    alphas = []
    soft_ranks = []
    min_svs = []
    Qs = []
    
    for im, m in enumerate(pretrained_model.modules()):
        Wmats = []
        #print(im, isinstance(m, nn.Conv2d))
        if linear and isinstance(m, nn.Linear):     
            W = np.array(m.weight.data.clone().cpu())
            Wmats = [W]
        elif conv2d and isinstance(m, nn.Conv2d):
            #print("conv2d")
            Wtensor = np.array(m.weight.data.clone().cpu())
            Wmats = get_conv2D_Wmats(Wtensor)
            
        #print("num mats ",len(Wmats))
        if Wmats is not None and len(Wmats)>0:
            for W in Wmats:
                #print(W.shape)

                M, N = np.min(W.shape), np.max(W.shape)
                Q=N/M 
                if M > 49:
                    sv, _ = RMT_Util.singular_spectrum(W)
                    evals = sv*sv
                    #scaled_evals = (1/N)*evals
                    # RMT_Util.fit_power_law
                    fit = powerlaw.Fit(evals, xmax=np.max(evals), verbose=False)  
                    alpha = fit.alpha
                    D = fit.D

                    dist = RMT_Util.best_dist(fit)
                    sigma = RMT_Util.fit_mp(evals, Q)
                    mp_soft_rank = RMT_Util.calc_mp_soft_rank(evals,Q,sigma)

                    min_svs.append(np.min(sv))

                    alphas.append(alpha)
                    soft_ranks.append(mp_soft_rank)
                    Qs.append(Q)
    return alphas, soft_ranks, min_svs, Qs


### PoweLaw fit for all FC and CNN layers

In [ ]:
all_alphas, all_ranks, all_min_svs, all_Qs = [], [], [], []
for model in tqdm([models.alexnet, 
              models.densenet121, models.densenet161, models.densenet169, models.densenet201, 
              models.inception_v3,
              models.resnet101, models.resnet152, models.resnet18, models.resnet34, models.resnet50, 
              models.squeezenet1_0, models.squeezenet1_1,
              models.vgg11, models.vgg11_bn,
              models.vgg16, models.vgg16_bn,
              models.vgg19, models.vgg19_bn ]):
    #print(model)
    #analyze_model(model, conv2d=True, plot=False)
    alphas, soft_ranks, min_svs, Qs = analyze_model(model)
    all_alphas.extend(alphas)
    all_ranks.extend(soft_ranks)
    all_min_svs.extend(min_svs)
    all_Qs.extend(Qs)

In [ ]:
plt.hist(all_alphas,bins=100);
plt.title(r"Power Law Exponents ($\alpha$) for PyTorch Conv2D Layers")
plt.show()

In [ ]:
plt.hist(all_ranks,bins=100);
plt.title(r"MP Soft Rank $\mathcal{R}_{mp}$ for PyTorch Conv2D Layers")
plt.show()

In [ ]:
aa = np.array(all_alphas)

In [ ]:
len(aa), len(aa[aa<5])

In [ ]:
100.0*len(aa[aa<5])/len(aa), 100.0*len(aa[aa<4])/len(aa),  100.0*len(aa[aa<1.5])/len(aa)

### Rank Loss

...only ~ 300/3000 ~ %13

This is hard to see in the plot

It may be easier to see on a log plot

In [ ]:
ms = np.array(all_min_svs)
len(ms[ms<0.00001]), len(ms), 100.0*len(ms[ms<0.00001])/len(ms)

In [ ]:
plt.hist(all_min_svs,bins=100);
plt.title(r"Min Singular Value PyTorch Conv2D Layers")
plt.show()

### Log Scale on Min Values

#TODO:  remove Q=1 cases

In [ ]:
lm = np.log10(all_min_svs)
plt.hist(lm,bins=100);
plt.title(r"$Log_{10}(\nu_{min})$ PyTorch Conv2D Layers")
plt.xlabel("Log10 Min Singular Value")
plt.savefig("img/log-min-nu-pytorch-conv2d")
plt.show()


In [ ]:
len(lm[lm<-6]), len(lm)